In [1]:
import cv2
import numpy as np
import os
import sys
sys.path.append('../../MyLibrary/')
import siannodel.img_process.opencv_extend as mycv
from easydict import EasyDict

## 数据生成测试

In [2]:
from data_gen import PhotoGenerator

In [3]:
gen_config = EasyDict({
    'warp_scale':0.1,
    'image_size':[768,1024],
    'img_dirs':[
        '/home/renhui/File/data/text_detection/train_data/data1_PNG/',
    ],
    'bg_img_dirs':[
        '/home/renhui/File/data/text_detection/background/',
    ],
})
photo_gen = PhotoGenerator(gen_config)

In [4]:
gen = photo_gen.gen()

In [5]:
tmp = gen.__next__()

1024 768


## network

In [3]:
import tensorflow as tf
import numpy as np
import cv2
import sys
sys.path.append('/home/renhui/File/MyLibrary/')
import siannodel.img_process.opencv_extend as mycv
import siannodel.ml.tf_estimator as myestimator
from siannodel.mylogger import *
from easydict import EasyDict
import siannodel.mytime as mytime

In [7]:
class Net(myestimator.BaseNet):
    def __init__(self, config):
        super(Net,self).__init__(config)
        
    def conv2d(self, input_tensor, filters, name, training):
        with tf.name_scope(name):
            output_tensor = tf.layers.conv2d(inputs=input_tensor,
                                             filters=filters,
                                             kernel_size=(3,3),
                                             padding='same',
                                             activation=None)
            output_tensor = tf.layers.batch_normalization(output_tensor,
                                                          training=training)
            output_tensor = tf.nn.relu(output_tensor)
        return output_tensor
    
    def upsample_and_concat(self, x1, x2, filters):
        deconv = tf.layers.conv2d_transpose(x1, filters, (2,2),
                                            strides=2)
        output_tensor = tf.concat([deconv,x2], 3)
        output_tensor.set_shape([None,None,None,filters*2])
        return output_tensor
    
    def __call__(self, x, training):
        network = {}
        network['conv1'] = self.conv2d(x, 32, 'conv1_1', training)
        network['conv1'] = self.conv2d(network['conv1'], 
                                       32, 'conv1_2', training)
        network['pool1'] = tf.layers.max_pooling2d(inputs=network['conv1'],
                                                   pool_size=[2,2], 
                                                   strides=2,
                                                   name='pool1')
        network['conv2'] = self.conv2d(network['pool1'], 64, 
                                       'conv2_1', training)
        network['conv2'] = self.conv2d(network['conv2'], 64, 
                                       'conv2_2', training)
        network['pool2'] = tf.layers.max_pooling2d(inputs=network['conv2'],
                                                   pool_size=[2,2], 
                                                   strides=2,
                                                   name='pool2')
        network['conv3'] = self.conv2d(network['pool2'], 128, 
                                       'conv3_1', training)
        network['conv3'] = self.conv2d(network['conv3'], 128, 
                                       'conv3_2', training)
        network['pool3'] = tf.layers.max_pooling2d(inputs=network['conv3'],
                                                   pool_size=[2,2], 
                                                   strides=2,
                                                   name='pool3')
        network['conv4'] = self.conv2d(network['pool3'], 256, 
                                       'conv4_1', training)
        network['conv4'] = self.conv2d(network['conv4'], 256, 
                                       'conv4_2', training)
        network['pool4'] = tf.layers.max_pooling2d(inputs=network['conv4'],
                                                   pool_size=[2,2], 
                                                   strides=2,
                                                   name='pool4')
    
        network['conv5'] = self.conv2d(network['pool4'], 512, 
                                       'conv5_1', training)
        network['conv5'] = self.conv2d(network['conv5'], 512, 
                                       'conv5_2', training)
        
        network['up6'] = self.upsample_and_concat(network['conv5'],
                                                  network['conv4'],
                                                  256)
        network['conv6'] = self.conv2d(network['up6'], 256, 
                                       'conv6_1', training)
        network['conv6'] = self.conv2d(network['conv6'], 256, 
                                       'conv6_2', training)
        
        network['up7'] = self.upsample_and_concat(network['conv6'],
                                                  network['conv3'],
                                                  128)
        network['conv7'] = self.conv2d(network['up7'], 128, 
                                       'conv7_1', training)
        network['conv7'] = self.conv2d(network['conv7'], 128, 
                                       'conv7_2', training)
        
        network['up8'] = self.upsample_and_concat(network['conv7'],
                                                  network['conv2'],
                                                  64)
        network['conv8'] = self.conv2d(network['up8'], 64, 
                                       'conv8_1', training)
        network['conv8'] = self.conv2d(network['conv8'], 64, 
                                       'conv8_2', training)
        
        network['up9'] = self.upsample_and_concat(network['conv8'],
                                                  network['conv1'],
                                                  32)
        network['conv9'] = self.conv2d(network['up9'], 32, 
                                       'conv9_1', training)
        network['conv9'] = self.conv2d(network['conv9'], 32, 
                                       'conv9_2', training)
        
        network['score_map'] = tf.layers.conv2d(network['conv9'], 1, (1,1),
                                                padding='same',
                                                activation=None,
                                                name='score_map')
        network['score_map'] = tf.nn.sigmoid(network['score_map'])
        
        network['flat'] = tf.layers.flatten(network['conv9'], name='flat')
        network['fc1'] = tf.layers.dense(inputs=network['flat'], units=1024,
                                         activation=tf.nn.relu, name='fc1')
        network['fc2'] =  tf.layers.dense(inputs=network['fc1'], units=256,
                                         activation=tf.nn.relu, name='fc2')
        network['M'] = tf.layers.dense(inputs=network['fc2'], units=8,
                                         activation=tf.nn.relu, name='M')
        network['pts'] = tf.layers.dense(inputs=network['fc2'], units=8,
                                         activation=tf.nn.relu, name='pts')
        return network

64.0

4